In [1]:
"|IMPORT PACKAGES|"
import numpy              as np
import pandas             as pd
import datetime
from   bokeh.plotting     import show, figure, output_file, save
from   bokeh.io           import show, output_notebook, curdoc, export_png
from   bokeh.models       import ColumnDataSource,LinearAxis, Range1d, NumeralTickFormatter, LabelSet, Label, BoxAnnotation, DatetimeTickFormatter, Text, Span
from   bokeh.models.tools import HoverTool
from   bokeh.transform    import dodge
from   datetime           import datetime as dt

In [2]:
"|IMPORT DATA|"
path = r'https://github.com/ncachanosky/research/blob/master/Economic%20Series/'
file = r'Resumen%20Estadistico%20-%20Argentina.xlsx?raw=true'
IO   = path + file

sheet = 'MONTHLY'

data = pd.read_excel(IO, sheet_name = sheet, usecols="B,BZ,CD,CH,CI", skiprows=2, nrows=404, engine='openpyxl') # Be patient...

data = data.rename(columns={"Inflation IPC Nucleo 12M"          :"IPC_N"})
data = data.rename(columns={"Inflation IPC 12M"                 :"IPC"})
data = data.rename(columns={"Inflation IPC-C 12M"               :"IPC_C"})
data = data.rename(columns={"Inflacion Promedio por presidencia":"PROMEDIO"})

data["IPC_N"]    = data["IPC_N"]   *100
data["IPC"]      = data["IPC"]     *100
data["IPC_C"]    = data["IPC_C"]   *100
data["PROMEDIO"] = data["PROMEDIO"]*100
    
data = data.drop(data.index[0:12])

data.loc[184:239,"NK"]    = data.iloc[184,4] # Promedio Néstor Kirchner
data.loc[240:335,"CFK"]   = data.iloc[240,4] # Promedio Cristina Kirchner
data.loc[336:383,"MM"]    = data.iloc[336,4] # Promedio Macri
data.loc[384:399,"AF"]    = data.iloc[384,4] # Promedio Alberto Fernández
data.loc[  0:227,"IPC_C"] = np.nan

In [3]:
"|CHECK DATA|"
data

,MONTH,IPC_N,IPC,IPC_C,PROMEDIO,NK,CFK,MM,AF
12,1989-01-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
13,1989-02-28,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
14,1989-03-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
15,1989-04-30,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
16,1989-05-31,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...
398,2021-03-31,45.559640,42.490362,42.490362,42.68443,NaN,NaN,NaN,42.68443
399,2021-04-30,49.710308,46.140361,46.140361,42.68443,NaN,NaN,NaN,42.68443
400,2021-05-31,52.510008,48.732013,48.732013,42.68443,NaN,NaN,NaN,NaN
401,2021-06-30,54.448063,50.187317,50.187317,42.68443,NaN,NaN,NaN,NaN


In [4]:
"|BUILD PLOT|"

cds = ColumnDataSource(data)

labels = ["Inflación Núcleo" ,
          "Inflación Oficial",
          "Inflación Congreso"]

text1 = Label(x=25, y=300, x_units='screen', y_units='screen', text_font_size="12px", text_color="darkslategrey",
              text="INFLACIÓN PROMEDIO ANUAL")

text2 = Label(x=25, y=285, x_units='screen', y_units='screen', text_font_size="12px", text_color="darkslategrey",
              text="Néstor Kirchner = 15.3%")

text3 = Label(x=25, y=270, x_units='screen', y_units='screen', text_font_size="12px", text_color="darkslategrey",
              text="Crsinta F. de Kirchner = 25.6%")

text4 = Label(x=25, y=255, x_units='screen', y_units='screen', text_font_size="12px", text_color="darkslategrey",
              text="Kirchnerismo = 20.4%")

text5 = Label(x=25, y=240, x_units='screen', y_units='screen', text_font_size="12px", text_color="darkslategrey",
              text="Mauricio Macri = 41.0%")

text6 = Label(x=25, y=225, x_units='screen', y_units='screen', text_font_size="12px", text_color="darkslategrey",
              text="Alberto Fernández = 42.7%")


#BCRA Presidents shades and names
PratGay     = BoxAnnotation(left=dt(2002,12,31).timestamp()*1000, right=dt(2004, 9,23).timestamp()*1000, line_color="black", fill_color="skyblue", fill_alpha=0.1)
Redrado     = BoxAnnotation(left=dt(2004, 9,24).timestamp()*1000, right=dt(2010, 1,10).timestamp()*1000, line_color="black", fill_color="purple" , fill_alpha=0.1)
delPont     = BoxAnnotation(left=dt(2010, 1,23).timestamp()*1000, right=dt(2013,11,18).timestamp()*1000, line_color="black", fill_color="green"  , fill_alpha=0.1)
Fabrega     = BoxAnnotation(left=dt(2013,11,19).timestamp()*1000, right=dt(2014,10, 1).timestamp()*1000, line_color="black", fill_color="orange" , fill_alpha=0.1)
Vanoli      = BoxAnnotation(left=dt(2014,10, 2).timestamp()*1000, right=dt(2015,12,10).timestamp()*1000, line_color="black", fill_color="black"  , fill_alpha=0.1)
Sturz       = BoxAnnotation(left=dt(2015,12,11).timestamp()*1000, right=dt(2018, 6,14).timestamp()*1000, line_color="black", fill_color="gold"   , fill_alpha=0.1)
Caputo      = BoxAnnotation(left=dt(2018, 6,15).timestamp()*1000, right=dt(2018, 9,25).timestamp()*1000, line_color="black", fill_color="gray"   , fill_alpha=0.1)
Sandleris   = BoxAnnotation(left=dt(2018, 9,26).timestamp()*1000, right=dt(2019,12,10).timestamp()*1000, line_color="black", fill_color="maroon" , fill_alpha=0.1)
Pesce       = BoxAnnotation(left=dt(2019,12,10).timestamp()*1000, right=dt(2021, 7,31).timestamp()*1000, line_color="black", fill_color="teal"   , fill_alpha=0.1)

PratGay_N   = Label(x=dt(2003,11, 2).timestamp()*1000, y=65, text="Prat-Gay"      , text_font_size="8pt", text_align="center", angle_units="deg", angle= 0)
Redrado_N   = Label(x=dt(2007, 5,25).timestamp()*1000, y=65, text="Redrado"       , text_font_size="8pt", text_align="center", angle_units="deg", angle= 0)
delPont_N   = Label(x=dt(2011,12,11).timestamp()*1000, y=65, text="Marcó del Pont", text_font_size="8pt", text_align="center", angle_units="deg", angle= 0)
Fabrega_N   = Label(x=dt(2014, 6,26).timestamp()*1000, y=67, text="Fábrega"       , text_font_size="8pt", text_align="right" , angle_units="deg", angle=90)
Vanoli_N    = Label(x=dt(2015, 7, 7).timestamp()*1000, y=67, text="Vanoli"        , text_font_size="8pt", text_align="right" , angle_units="deg", angle=90)
Sturz_N     = Label(x=dt(2017, 5,12).timestamp()*1000, y=67, text="Sturzenneger"  , text_font_size="8pt", text_align="right" , angle_units="deg", angle=90)
Caputo_N    = Label(x=dt(2018,10, 4).timestamp()*1000, y=67, text="Caputo"        , text_font_size="8pt", text_align="right" , angle_units="deg", angle=90)
Sandleris_N = Label(x=dt(2019, 7, 3).timestamp()*1000, y=67, text="Sandleris"     , text_font_size="8pt", text_align="right" , angle_units="deg", angle=90)
Pesce_N     = Label(x=dt(2021, 2, 3).timestamp()*1000, y=67, text="Pesce"         , text_font_size="8pt", text_align="right" , angle_units="deg", angle=90)


#BUILD FIGURE
p = figure(title = "EL HUB ECONÓMICO | INFLACIÓN INTERANUAL",
           y_range      = (-10, 70),
           x_range      = (dt(1999, 1,31).timestamp()*1000, dt(2021, 8,31).timestamp()*1000),
           plot_height  = 400,
           plot_width   = 700)

p.xaxis[0].formatter = DatetimeTickFormatter(months="%b %Y")
p.toolbar_location = "right"
p.toolbar.autohide = True

#AXIS 1 (LEFT)
#Inflación interanual
p1 = p.line("MONTH", "IPC_N", color="green", line_alpha=0.75, width=2, legend_label=labels[0], source=cds, muted_alpha=0.20)
p2 = p.line("MONTH", "IPC"  , color="black", line_alpha=0.75, width=2, legend_label=labels[1], source=cds, muted_alpha=0.20)
p3 = p.line("MONTH", "IPC_C", color="blue" , line_alpha=0.75, width=2, legend_label=labels[2], source=cds, muted_alpha=0.20)

#Inflación promedio
p4 = p.line("MONTH", "NK" , color="red", line_alpha=0.75, width=2, line_dash=[4,4], source=cds)
p5 = p.line("MONTH", "CFK", color="red", line_alpha=0.75, width=2, line_dash=[4,4], source=cds)
p6 = p.line("MONTH", "MM" , color="red", line_alpha=0.75, width=2, line_dash=[4,4], source=cds)
p7 = p.line("MONTH", "AF" , color="red", line_alpha=0.75, width=2, line_dash=[4,4], source=cds)

p.add_tools(HoverTool(renderers=[p1], tooltips = [("Inflación Núcleo"  , "@IPC_N{0.0}")], mode="vline"))
p.add_tools(HoverTool(renderers=[p2], tooltips = [("Inflación Oficial" , "@IPC{0.0}")]  , mode="vline"))
p.add_tools(HoverTool(renderers=[p3], tooltips = [("Inflación Congreso", "@IPC_C{0.0}")], mode="vline"))

p.add_layout(text1)
p.add_layout(text2)
p.add_layout(text3)
p.add_layout(text4)
p.add_layout(text5)
p.add_layout(text6)

#ADD BCRA PRESIDENTS
p.add_layout(PratGay)
p.add_layout(Redrado)
p.add_layout(delPont)
p.add_layout(Fabrega)
p.add_layout(Vanoli)
p.add_layout(Sturz)
p.add_layout(Caputo)
p.add_layout(Sandleris)
p.add_layout(Pesce)

p.add_layout(PratGay_N)
p.add_layout(Redrado_N)
p.add_layout(delPont_N)
p.add_layout(Fabrega_N)
p.add_layout(Vanoli_N)
p.add_layout(Sturz_N)
p.add_layout(Caputo_N)
p.add_layout(Sandleris_N)
p.add_layout(Pesce_N)

#LEGEND
p.xaxis[0].formatter = DatetimeTickFormatter(months="%m %Y")
h_axis = Span(location=0, dimension="width", line_color="black")
p.add_layout(h_axis)
p.legend.location     = "bottom_right"
p.legend.orientation  = "horizontal"
p.legend.click_policy = "mute"
output_notebook()
show(p)

In [ ]:
"|EXPORT .PNG FILE|"

export_png(p, filename="inflacion_interanual.png")

In [ ]:
"|# CREATE HTML FILE|"

output_file(filename="inflacion_interanual.html", title="Deuda pública")
save(p)

In [5]:
"|CREATE JSON FILE|"
import json
import bokeh.embed

from bokeh.embed import json_item

j = json.dumps(json_item(p, "inflacion_interanual"))
with open("inflacion_interanual.json", "w") as fp:
    json.dump(j, fp)